In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [2]:
tf.__version__

#텐서플로우 버전 2.3.0

'2.3.0'

In [3]:
device = tf.device('/gpu:0')
# 디바이스 gpu 0버전

In [4]:
train_path = './drive/My Drive/00.data/train/'
test_path = './drive/My Drive/00.data/test/'

#코랩기준 테스트 경로와 훈련 경로

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


x_train_datagen = ImageDataGenerator(
                rotation_range=0.2,
                rescale= 1./255.,
                zoom_range=0.2,
                horizontal_flip=True,)
y_train_datagen = ImageDataGenerator(
                rotation_range=0.2,
                rescale= 1./255.,
                shear_range=0.05,
                zoom_range=0.2,
                horizontal_flip=True,)
#
x_train_generator = image_datagen.flow_from_directory(
        train_path,
        classes = ['img'],
        class_mode = None,
        target_size = (128,128),
        batch_size = 16,
        seed = 1)
#
y_train_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = ['mask'],
        class_mode = None,
        target_size = (128,128),
        batch_size = 16,
        seed = 1)

trainSet_generator = zip(image_generator, mask_generator)

Found 2520 images belonging to 1 classes.
Found 2520 images belonging to 1 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


In [16]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model

#first_block
def unet(weight = None,input_shape = (128,128,3),initial_filter = 16):
    inputs = Input(input_shape)
    conv1 = Conv2D(initial_filter , (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1_1 = Conv2D(initial_filter , (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPool2D(pool_size = (2,2))(conv1_1)

    conv2 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2_1 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPool2D(pool_size = (2,2))(conv2_1)

    conv3 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3_1 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPool2D(pool_size = (2,2))(conv3_1)

    conv4 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4_1 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    pool4 = MaxPool2D(pool_size = (2,2))(conv4_1)

    ####################################downsampling#############################################

    conv5 = Conv2D(initial_filter * 16, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5_1 = Conv2D(initial_filter * 16, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    up5 = UpSampling2D(size = (2,2))(conv5_1)

    ####################################upsampling#############################################

    merge_4 = concatenate([conv4_1,up5], axis = 3) 
    conv_4 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge_4)
    conv_4_1 = Conv2D(initial_filter * 8, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv_4)
    up4 = UpSampling2D(size = (2,2))(conv_4_1)

    merge_3 = concatenate([conv3_1,up4], axis = 3)
    conv_3 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge_3)
    conv_3_1 = Conv2D(initial_filter * 4, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv_3) 
    up3 = UpSampling2D(size = (2,2))(conv_3_1)

    merge_2 = concatenate([conv2_1,up3], axis = 3)
    conv_2 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge_2)
    conv_2_1 = Conv2D(initial_filter * 2, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv_2) 
    up2 = UpSampling2D(size = (2,2))(conv_2_1)

    merge_1 = concatenate([conv1_1,up2], axis = 3)
    conv_1 = Conv2D(initial_filter, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge_1)
    conv_1_1 = Conv2D(initial_filter, (3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv_1)

    out = Conv2D(1, (1,1))(conv_1_1)

    model = Model(inputs,out)

    #model = multi_gpu_model(model,gpus =2)
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    if weight:
        model.load_weights(weight)
    
    return model

In [18]:
model = unet()

In [19]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 128, 128, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 128, 128, 16) 2320        conv2d_19[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 64, 64, 16)   0           conv2d_20[0][0]                  
_______________________________________________________________________________________

In [ ]:
import math
history = model.fit_generator(train_generator,25,epochs = 5)

Epoch 1/5
 6/25 [======>.......................] - ETA: 6:34 - loss: 2.9836 - accuracy: 0.7950